In [1]:
import openturns as ot
import numpy as np
import otbenchmark as otb
import pandas as pd

In [2]:
p = otb.OTReliabilityAlgorithmBenchmark()
list = p.problemslist

In [3]:
k = 18

In [4]:
name = list[k].getName()
name

'RP111'

In [5]:
N = 10 ** 9
N

1000000000

In [6]:
name = list[k].getName()

resultsFORM = otb.OTReliabilityAlgorithmBenchmark.FORM(
        list[k],
        nearestPointAlgo="Cobyla",
        maximumEvaluationNumber=100,
        maximumAbsoluteError=1.0e-3,
        maximumRelativeError=1.0e-3,
        maximumResidualError=1.0e-3,
        maximumConstraintError=1.0e-3)

resultsSORM = otb.OTReliabilityAlgorithmBenchmark.SORM(
        list[k],
        nearestPointAlgo="Cobyla",
        maximumEvaluationNumber=100,
        maximumAbsoluteError=1.0e-3,
        maximumRelativeError=1.0e-3,
        maximumResidualError=1.0e-3,
        maximumConstraintError=1.0e-3)

resultsMC = otb.OTReliabilityAlgorithmBenchmark.MonteCarloSampling(list[k], maximumOuterSampling=N,
                                                                   coefficientOfVariation=0.1, blockSize=2)

resultFORMIS = otb.OTReliabilityAlgorithmBenchmark.FORMImportanceSampling(list[k],
        nearestPointAlgo="Cobyla",
        maximumEvaluationNumber=100,
        maximumAbsoluteError=1.0e-3,
        maximumRelativeError=1.0e-3,
        maximumResidualError=1.0e-3,
        maximumConstraintError=1.0e-3,
        maximumOuterSampling=N,
        coefficientOfVariation=0.1)

resultSubset = otb.OTReliabilityAlgorithmBenchmark.SubsetSampling(
        list[k], maximumOuterSampling=10000, coefficientOfVariation=0.1, blockSize=1)

result = [resultsMC, resultsFORM, resultsSORM, resultFORMIS, resultSubset]

In [7]:
a = "[%.5f, %.5f] " % ( resultsMC[4], resultsMC[5])


b = "[%.5f, %.5f] " % ( resultFORMIS[4], resultFORMIS[5])


c = "[%.5f, %.5f] " % ( resultSubset[4], resultSubset[5])


In [8]:
mc1 = np.format_float_scientific(np.float32(resultsMC[4]))
mc2 = np.format_float_scientific(np.float32(resultsMC[5]))

ic1 = "["+mc1+", "+mc2+"]"
ic1

'[6.0349527e-07, 8.977299e-07]'

In [9]:
formis1 = np.format_float_scientific(np.float32(resultFORMIS[4]))
formis2 = np.format_float_scientific(np.float32(resultFORMIS[5]))

ic2 = "["+formis1+", "+formis2+"]"
ic2

'[1.6309134e-07, 2.4240524e-07]'

In [10]:
subset1 = np.format_float_scientific(np.float32(resultSubset[4]))
subset2 = np.format_float_scientific(np.float32(resultSubset[5]))

ic3 = "["+subset1+", "+subset2+"]"

ic3

'[6.0968114e-07, 9.8752e-07]'

In [11]:
M = [[None]*5]*6

M[2][3] = a

M

[[None, None, None, '[0.00000, 0.00000] ', None],
 [None, None, None, '[0.00000, 0.00000] ', None],
 [None, None, None, '[0.00000, 0.00000] ', None],
 [None, None, None, '[0.00000, 0.00000] ', None],
 [None, None, None, '[0.00000, 0.00000] ', None],
 [None, None, None, '[0.00000, 0.00000] ', None]]

In [12]:
l = np.array([[1, 1, 1, ic3, 1, a], [1, 1, 1, ic3, 1, a], [1, 1, 1, ic3, 1, a], [1, 1, 1, ic3, 1, a],[1, 1, 1, ic3, 1, a]])

In [13]:

method = ["CMC", "FORM", "SORM", "FORM-IS", "SUBSET"]
metrics = ["Probabilité de référence de "+name, "Probabilité estimée", "Erreur absolue","Intervalle de confiance",
           "nombre d'évaluations", "Log erreur relative"]

results = l



for i in range(len(method)):
    results[i][0] =  np.format_float_scientific(np.float32(list[k].getProbability()))

for i in range(len(method)):
    results[i][1] =  np.format_float_scientific(np.float32(result[i][0]))

for i in range(len(method)):
    results[i][2] =  np.format_float_scientific(np.float32(result[i][1]))

for i in range(len(method)):
    results[i][4] =  result[i][3]
    
for i in range(len(method)):
    results[i][5] =  "%.2f " % (result[i][2])


results[0][3] = ic1
results[1][3] = "-----------------"
results[2][3] = "-----------------"
results[3][3] = ic2
results[4][3] = ic3


df = pd.DataFrame(results, index = method, columns=metrics)
print("Function g : ", list[k].getEvent().getFunction())
df

Function g :  [x1,x2]->[12.5 - abs(x1 * x2)]


,Probabilité de référence de RP111,Probabilité estimée,Erreur absolue,Intervalle de confiance,nombre d'évaluations,Log erreur relative
CMC,7.8e-07,7.5061257e-07,2.938744e-08,"[6.0349527e-07, 8.977299e-0",199836784,1.42
FORM,7.8e-07,1.7941125e-02,1.7940344e-02,-----------------,101,-4.36
SORM,7.8e-07,1.2687665e-02,1.2686885e-02,-----------------,101,-4.21
FORM-IS,7.8e-07,2.027483e-07,5.772517e-07,"[1.6309134e-07, 2.4240524e-",81288,0.13
SUBSET,7.8e-07,7.986006e-07,1.86006e-08,"[6.0968114e-07, 9.8752e-07]",70000,1.62


In [14]:
df.to_csv('problem111.csv')